# WEEK2           
> ## 词汇表征(Word representation)                  

One-hot编码的缺陷:模型的泛化能力会因为One-hot编码后各类词的内积都为0所以**认为同类词毫无关联**，比如apple与orange     

**解决方法——词嵌入(word embeddings)**:利用特征化的方式表示各个词语     
列索引为特征，行索引为词语，值表示词语在与这个特征的关联度     
<img style="float: center;" src="course_5_pics/WEEK_2_1.PNG" width=600 height=600>

> ## 使用词嵌入   

1. 利用网络上或等等渠道获得的大量文本学习词嵌入(下载预训练过的词嵌入模型)    
2. 将词嵌入模型迁移到具有少量标记数据的网络模型中  
3. 可选择的：利用已有的数据微调已获得的词嵌入模型     

> ## 词嵌入的特性     

词嵌入能够实现词语间的类比推理     
<img style="float: center;" src="course_5_pics/WEEK_2_2.PNG" width=600 height=600>       
```
需要推理Man-Woman&King-Queen
```
在词嵌入模型中，将Man与Woman的嵌入向量命名为$e_{Man}$与$e_{Woman}$，那么有向量运算:
$$e_{Man}-e_{Woman}=e_{King}-e_{Queen}$$
两对词语的词嵌入向量在Gender维度上距离相同。    

类比推理可以理解为(实际应用中，该方法准确性不高):        
$$Find\ \ word\ \ w:arg\underset{{w}}{max}\ \ similarity(e_w,e_{Woman}-e_{Man}+e_{King})$$    

一种相似度函数——余弦相似度:$sim(u,v)=\cfrac{u^Tv}{||u||_2||v||_2}$      

> ## 嵌入矩阵   

当学习词嵌入模型时，实际上是在学习一个**嵌入矩阵**       
<img style="float: center;" src="course_5_pics/WEEK_2_3.PNG" width=600 height=600>      
一个词嵌入矩阵$E$与一个词$j$的One-hot编码相乘实际上得到的是词嵌入矩阵中关于$j$的那一列$e_j$，在实际操作中，通常直接提取。

> ## 学习词嵌入     

通过建立一个语言模型的方法将嵌入矩阵与词One-hot编码相乘传入神经网络进行训练来训练词嵌入模型       
网络会学到将类似词语的特征向量相似的时候结果会最优化
<img style="float: center;" src="course_5_pics/WEEK_2_4.PNG" width=600 height=600>     

不同的选择上下文的方法:     
1. 前四个单词   
2. 前四个与后四个单词  
3. 上一个单词   
4. 邻近的一个单词(skip grams)    

当学习语言模型时——方法1，学习词嵌入时——方法1、2、3
<img style="float: center;" src="course_5_pics/WEEK_2_4.PNG" width=600 height=600>     

> ## Word2Vec    

Word2Vec模型中，主要有Skip-Gram和CBOW两种模型，从直观上理解，Skip-Grams是**给定input word来预测上下文**。而CBOW是给定上下文，来预测input word。   
```
例:i want a glass of orange juice to go along with my cereal
context   target

orange    juice
orange    glass
orange    my
```
这是一个监督学习模型，但不是为了学习这个模型，而是利用这个监督学习来学习词嵌入模型    

### Word2Vec目标
Word2Vec模型实际上分为了两个部分。第一部分为建立模型，第二部分是通过模型获取嵌入词向量。      
Word2Vec的整个建模过程实际上与自编码器（auto-encoder）的思想很相似，即先基于训练数据构建一个神经网络，当这个模型训练好以后，并不会用这个训练好的模型处理新的任务，真正需要的是这个模型通过训练数据所学得的参数，例如**隐层的权重矩阵——这些权重在Word2Vec中实际上就是我们试图去学习的“word vectors”**。    
### Word2Vec过程     
```
例:i want a glass of orange juice to go along with my cereal
```

1. 首先在一个句子中选择一个input word，定义一个**窗口大小**(左右侧几个单词作为选择输出单词的界限)与一个**选择个数**(在窗口中选择的单词个数).组成**单词对(input_word,ouput_word)**   
<img style="float: center;" src="course_5_pics/WEEK_2_6.PNG" width=600 height=600>     

2. 对每个**输入单词与输出单词进行One-hot编码**，输入单词输入隐层神经元(没有激活函数，线性相乘)——参数即**嵌入矩阵**，最后输入softmax得出结果，利用输出单词进行参数训练。   
<img style="float: center;" src="course_5_pics/WEEK_2_7.PNG" width=600 height=600>

3. 具体如下:    
context:$c$     
target:$t$    
将输入单词context$c$进行one-hot编码得到$O_c$     
$O_c$乘以嵌入矩阵$E$,得到$e_c=EO_c$    
$e_c$输入softmax层得到目标单词的预测向量$\hat y$      

其中:Softmax结果——$p(t|c)=\cfrac{e^{\theta_t^Te_c}}{\sum_j^v\theta_j^Te_c}$,$v$为词汇表长度，$\theta_t$为单词$t$有关一个参数         
该方法主要的难点在于，语料库过长，最后在求softmax层时需要很大的算例，所以通常使用分级softmax。
### 直觉上的理解

如果两个不同的单词有着非常相似的“上下文”（也就是**窗口单词**很相似，比如“Kitty climbed the tree”和“Cat climbed the tree”），那么通过模型训练，这两个单词的**嵌入向量将非常相似**。       

这种方法实际上也可以帮助你进行词干化(stemming)(词干化(stemming)就是去除词缀得到词根的过程。)，例如，神经网络对”ant“和”ants”两个单词会习得相似的词向量。

> ## 负采样     

```
例:i want a glass of orange juice to go along with my cereal
```

### 负采样的训练流程

1. 首先选取一个词语与其上下文，该例子中选择```orange-juice```，作为一个**正样本**，目标值为**1**。然后从词汇表中随机选择$k$(小数据集：k=5-20；大数据集:k=2-5)个词语与在样本中选择的词语```orange```组成**负样本**，目标值为**0**(随机选择的词语在上下文中也标记为负样本)     

<img style="float: center;" src="course_5_pics/WEEK_2_8.PNG" width=200 height=200>    

```
x(c,t)             y

(orange,juice)     1
(orange,king)      0
(orange,book)      0
(orange,the)       0
(orange,of)        0

```

2. 利用Logistics回归训练，$P(1|c,t)=\sigma(\theta_t^Te_c)$,神经网络图如下:    
<img style="float: center;" src="course_5_pics/WEEK_2_9.PNG" width=500 height=500>     
先将词语$c$进行one-hot编码，然后乘以嵌入矩阵$E$得到$e_c$，最后输入到宽度为**单词列表长度**的二分类层当中，每个逻辑回归(神经元)输出一个结果。这样每批样本只需要训练$k+1$个词语对应的神经元的二元分类问题(**k个负样本和1个正样本**)

### 如何进行负采样     
1. 通过词频(单词的经验频率)$P(w_i)=f(w_i)$进行采样(某些词的词频特别高,比如一些代词，介词等等)   
2. 通过词汇表的均匀分布进行随机采样(缺乏代表性)    

**解决方法**:$P(w_i) = \cfrac{f(w_i)^{\frac{3}{4}}}{\sum_j^vf(w_j)^{\frac{3}{4}}}$     

> ## Glove算法(Glove:Global vector for word representation)     

$c,t$是上下文中的词语与目标词汇    
设$x_{ij}$是词语$i$在词语$j$的上下文出现的次数($i-t,j-c$)    
$x_{ij}=x_{ji}$在某些上下文定义中不成立，例如$c$是$t$的上一个单词    

在Glove算法中，上下文的定义是一个左右长度相等的窗口，因此$x_{ij}=x_{ji}$

### Glove算法模型:    
$$minimize\ \ \sum_i^v\sum_j^vf(x_{ij})(\theta_i^Te_j+b_i+b_j-Log(x_{ij}))$$       

**权重项$f(x_{ij})$的作用**:         

1. $f(x_{ij})=0,if\ \ x_{ij}=0(log(x_{ij})\rightarrow \infty)$，这里认为$0log0=0$    
2. $f(x_{ij})$函数的选择可以令词频较高的词语不会获得那么大的权重(the,of,a一类的代词以及介词)，也让一些词频低的词不会获得那么小的权重(durion等)     

### 注意事项     
嵌入矩阵所代表的轴可能无法理解，甚至相互之间不正交     

>## 情绪分类     

例子:    
<img style="float: center;" src="course_5_pics/WEEK_2_10.PNG" width=500 height=500>      
给定一个句子为输入值，输出值为评价星级。     

### DNN方法:     
<img style="float: center;" src="course_5_pics/WEEK_2_11.PNG" width=500 height=500>        
将每个词语进行One-hot编码后于嵌入矩阵相乘得到词嵌入向量，然后输入一个平均值运算，最后输入一个星级的softmax分类器。     

平均值运算会将所有词的评价程度平均后得出每个星级评价的概率值。但存在一个缺陷——没有考虑语序，当遇到左下角的句子出现了很多个good时会存在偏差。     

### RNN方法     
<img style="float: center;" src="course_5_pics/WEEK_2_12.PNG" width=500 height=500>      

> ## 词嵌入除偏(性别、种族的偏见而不是bias)    

<img style="float: center;" src="course_5_pics/WEEK_2_13.PNG" width=500 height=500>      
训练过的词嵌入模型可能会显示出性别、种族等的歧视(如上图)     

### 减少偏见的程序(以性别为例):   
**1. 鉴定我们需要减少的偏见**     
$e_{man}-e_{female}$          
$e_{boy}-e_{girl}$    
$\vdots$    
$求出以上差的平均值$

<img style="float: center;" src="course_5_pics/WEEK_2_14.PNG" width=200 height=200>    
横轴是偏见趋势，纵轴是无偏见趋势(实际情况一般大于一维)      

**2. 中和步骤:对于每一个非定义性的单词，消除偏见**     
比如babysister,computer programmer等没有明确的性别含义，向非偏见轴零点移动

**3. 均衡步骤**        
让grandmother-grandfather等词语于非偏见轴保持等距离